In [4]:
import pandas as pd # $ pip install pandas
import folium # conda install -c conda-forge folium
# import chart_studio.plotly as py # conda install -c plotly plotly, conda install -c plotly chart-studio
import plotly.graph_objects as go
import plotly.express as px
import os

# Standorte

In [5]:
m = folium.Map(location=[51.2099, 10.4571], zoom_start=6)
folium.Marker([52.461300, 13.335307], tooltip='	Neue Schönhauser Straße, Berlin').add_to(m)
folium.Marker([53.07736868080147, 8.804847969315563], tooltip="Obernstraße, Bremen").add_to(m)
folium.Marker([51.047494158917196, 13.737369656599013], tooltip="dresden-prager straße").add_to(m)
folium.Marker([551.222185050102674, 6.779255270096054], tooltip="düsseldorf-königsallee ostseiten").add_to(m)
folium.Marker([50.97797259773246, 11.026376827762066], tooltip="erfurt-marktstraßen").add_to(m)
folium.Marker([50.11494292444826, 8.687410770070198], tooltip="frankfurt a.m.-zeil").add_to(m)
folium.Marker([51.480738258522834, 11.97536492962145], tooltip="halle (saale)-leipziger straße").add_to(m)
folium.Marker([53.55376220607088, 9.991887654803696], tooltip="hamburg-jungfernstieg").add_to(m)
folium.Marker([52.374956125557894, 9.737387597107245], tooltip="hannover-georgstraße").add_to(m)
folium.Marker([54.3211874676847, 10.13512538365591], tooltip="kiel-holstenstraße").add_to(m)
folium.Marker([50.001027164285354, 8.270445585412894], tooltip="mainz-stadthausstraße").add_to(m)
folium.Marker([48.137685457750635, 11.573367770032743], tooltip="münchen-kaufingerstraße").add_to(m)
folium.Marker([54.088503508553394, 12.134802012486357], tooltip="rostock-kröpeliner straße").add_to(m)
folium.Marker([49.2359910863105, 6.993367970054943], tooltip="saarbrücken-bahnhofstraße").add_to(m)
folium.Marker([48.77820910005942, 9.177711579841167], tooltip="stuttgart-königstraßee").add_to(m)

m

# Daten einlesen

In [6]:
files = [ 'data/' + x for x in os.listdir('data') ]
files

['data/berlin-neue schönhauser straße-20210817-20221031-hour.csv',
 'data/bremen-obernstraße-20190930-20221031-hour.csv',
 'data/dresden-prager straße (nord)-20190728-20221031-hour.csv',
 'data/düsseldorf-königsallee ostseite (süd)-20180430-20221031-hour.csv',
 'data/erfurt-marktstraße-20220720-20221031-hour.csv',
 'data/frankfurt a.m.-zeil (mitte)-20190630-20221031-hour.csv',
 'data/halle (saale)-leipziger straße (west)-20201209-20221031-hour.csv',
 'data/hamburg-jungfernstieg-20180927-20221031-hour.csv',
 'data/hannover-georgstraße-20180731-20221031-hour.csv',
 'data/kiel-holstenstraße (mitte)-20200505-20221031-hour.csv',
 'data/mainz-stadthausstraße-20180430-20221031-hour.csv',
 'data/münchen-kaufingerstraße-20190827-20221031-hour.csv',
 'data/rostock-kröpeliner straße (west)-20191031-20221031-hour.csv',
 'data/saarbrücken-bahnhofstraße (mitte)-20180430-20221031-hour.csv',
 'data/stuttgart-königstraße (mitte)-20180430-20221031-hour.csv']

In [7]:
data = [ pd.read_csv(x, delimiter=';') for x in files ] # Stadt1 in data[0], Stadt2 in data[1] ...
data[1].head(1)

,location,time of measurement,weekday,pedestrians count,temperature in ºc,weather condition,incidents
0,"Obernstraße, Bremen",2019-10-01 00:00:00 +0200,Tuesday,16,NaN,NaN,NaN


# Datensatz aufbereiten

"time of measurement" in seine Bestandteile Jahr, Monat, Tag, Stunde auflösen.

In [8]:
for i in range(len(data)):
    # Spalte "time of measurement" auflösen
    data[i][['date','time', 'timezone']]= data[i]["time of measurement"].str.split(" ",expand=True)
    data[i][['year', 'month', 'day']] = data[i]['date'].str.split("-", expand=True)
    data[i][['hour', 'minute', 'second']] = data[i]['time'].str.split(":", expand=True)
    
    # Spalten umbenennen
    data[i] = data[i].rename(columns={"pedestrians count" : "pedestrians", "temperature in ºc" : "temperature"})
    
    # Alle temprären Spalten löschen
    data[i].drop(columns=['time of measurement', 'date', 'time', 'timezone', 'minute', 'second'], inplace=True)
    
    data[i] = data[i].astype({'year' : 'int16', 'month' : 'int16', 'day' : 'int16', 'hour' : 'int16', 'incidents' : 'int16', 'temperature' : 'int16', 'pedestrians' : 'int32'}, errors='ignore')
    
data[0]

,location,weekday,pedestrians,temperature,weather condition,incidents,year,month,day,hour
0,"Neue Schönhauser Straße, Berlin",Wednesday,31,15,partly-cloudy-night,NaN,2021,8,18,0
1,"Neue Schönhauser Straße, Berlin",Wednesday,13,14,partly-cloudy-night,NaN,2021,8,18,1
2,"Neue Schönhauser Straße, Berlin",Wednesday,0,14,partly-cloudy-night,NaN,2021,8,18,2
3,"Neue Schönhauser Straße, Berlin",Wednesday,4,14,partly-cloudy-night,NaN,2021,8,18,3
4,"Neue Schönhauser Straße, Berlin",Wednesday,3,13,partly-cloudy-night,NaN,2021,8,18,4
...,...,...,...,...,...,...,...,...,...,...
10556,"Neue Schönhauser Straße, Berlin",Monday,529,15,cloudy,NaN,2022,10,31,19
10557,"Neue Schönhauser Straße, Berlin",Monday,287,14,cloudy,NaN,2022,10,31,20
10558,"Neue Schönhauser Straße, Berlin",Monday,204,14,cloudy,NaN,2022,10,31,21
10559,"Neue Schönhauser Straße, Berlin",Monday,140,13,cloudy,NaN,2022,10,31,22


# Daten analysieren

In [9]:
def sumOfYear(data, year):
    result = []
    for city in data:
        filteredByYear = city.loc[city['year'] == year]
        result.append((city['location'][0], filteredByYear['pedestrians'].sum()))
    return result

sumOfYear(data, 2021)

[('Neue Schönhauser Straße, Berlin', 1189810),
 ('Obernstraße, Bremen', 4923375),
 ('Prager Straße (Nord), Dresden', 8298939),
 ('Königsallee Ostseite (Süd), Düsseldorf', 3579864),
 ('Marktstraße, Erfurt', 0),
 ('Zeil (Mitte), Frankfurt a.M.', 12529295),
 ('Leipziger Straße (West), Halle (Saale)', 5496608),
 ('Jungfernstieg, Hamburg', 7379766),
 ('Georgstraße, Hannover', 14034597),
 ('Holstenstraße (Mitte), Kiel', 5033672),
 ('Stadthausstraße, Mainz', 3215351),
 ('Kaufingerstraße, München', 15517650),
 ('Kröpeliner Straße (West), Rostock', 6840266),
 ('Bahnhofstraße (Mitte), Saarbrücken', 9839731),
 ('Königstraße (Mitte), Stuttgart', 7581398)]

In [10]:
def sumOfYearPerWeekday(data, year):
    result = [
        
    ]
    weekdays = [
        'Monday',
        'Tuesday',
        'Wednesday',
        'Thursday',
        'Friday',
        'Saturday',
        'Sunday'
    ]
    for city in data:
        filteredByYear = city.loc[city['year'] == year]
        for day in weekdays:
            filteredByWeekday = filteredByYear.loc[city['weekday'] == day]
            result.append((city['location'][0], day, filteredByWeekday['pedestrians'].sum()))

    return result

sumOfYearPerWeekday(data, 2021)

[('Neue Schönhauser Straße, Berlin', 'Monday', 159968),
 ('Neue Schönhauser Straße, Berlin', 'Tuesday', 156987),
 ('Neue Schönhauser Straße, Berlin', 'Wednesday', 168129),
 ('Neue Schönhauser Straße, Berlin', 'Thursday', 169726),
 ('Neue Schönhauser Straße, Berlin', 'Friday', 192303),
 ('Neue Schönhauser Straße, Berlin', 'Saturday', 262617),
 ('Neue Schönhauser Straße, Berlin', 'Sunday', 80080),
 ('Obernstraße, Bremen', 'Monday', 716665),
 ('Obernstraße, Bremen', 'Tuesday', 720608),
 ('Obernstraße, Bremen', 'Wednesday', 715667),
 ('Obernstraße, Bremen', 'Thursday', 697398),
 ('Obernstraße, Bremen', 'Friday', 776427),
 ('Obernstraße, Bremen', 'Saturday', 1036640),
 ('Obernstraße, Bremen', 'Sunday', 259970),
 ('Prager Straße (Nord), Dresden', 'Monday', 1153268),
 ('Prager Straße (Nord), Dresden', 'Tuesday', 1170724),
 ('Prager Straße (Nord), Dresden', 'Wednesday', 1209002),
 ('Prager Straße (Nord), Dresden', 'Thursday', 1154988),
 ('Prager Straße (Nord), Dresden', 'Friday', 1294519),
 ('

# Daten visualisieren

In [11]:
absPed2020 = sumOfYear(data, 2020)
fig = px.bar(absPed2020, x=[x[0] for x in absPed2020], y=[y[1] for y in absPed2020], title='Absolute Zahlen 2020', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig.show()

In [12]:
absPed2021 = sumOfYear(data, 2021)
fig = px.bar(absPed2021, x=[x[0] for x in absPed2021], y=[y[1] for y in absPed2021], title='Absolute Zahlen 2021', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig.show()

In [13]:
absPed2022 = sumOfYear(data, 2022)
fig2 = px.bar(absPed2022, x=[x[0] for x in absPed2022], y=[y[1] for y in absPed2022], title='Absolute Zahlen 2022', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig2.show()

In [14]:
# Location mit Fußgänger auf Wochen

weeklyPed2022 = sumOfYearPerWeekday(data, 2022)
fig2 = px.scatter_3d(weeklyPed2022, x=[x[0] for x in weeklyPed2022], y=[y[1] for y in weeklyPed2022], z=[z[2] for z in weeklyPed2022], title='Absolute Zahlen 2022', labels={'x' : 'Ort', 'y' : 'Anzahl Fußübergänger'})
fig2.show()

In [21]:
weeklyPed2022 = sumOfYearPerWeekday(data, 2022)


long_df = pd.DataFrame(weeklyPed2022, columns=['location', 'weekday', 'countp'])
print(long_df)
fig = px.bar(long_df, x='weekday', y='countp', color="location")
fig.show()

                            location    weekday   countp
0    Neue Schönhauser Straße, Berlin     Monday   304035
1    Neue Schönhauser Straße, Berlin    Tuesday   297457
2    Neue Schönhauser Straße, Berlin  Wednesday   301900
3    Neue Schönhauser Straße, Berlin   Thursday   301091
4    Neue Schönhauser Straße, Berlin     Friday   348506
..                               ...        ...      ...
100   Königstraße (Mitte), Stuttgart  Wednesday  1534329
101   Königstraße (Mitte), Stuttgart   Thursday  1502002
102   Königstraße (Mitte), Stuttgart     Friday  1829994
103   Königstraße (Mitte), Stuttgart   Saturday  3313623
104   Königstraße (Mitte), Stuttgart     Sunday  1093335

[105 rows x 3 columns]


## Winter Sommer Jahreszeiten

## Durchschnitte (Jahr, Monat, Tag)

### Große Events vgl. zu normalen Tagen (einem durschnittstag) 

### Corona vergleich

## Wetter (Bevölk, Regen)